In [1]:
!pip install altair
!pip install altair vega_datasets
!pip install vega
!pip install altair_viewer
!pip install textwrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 22.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple

In [2]:
import numpy as np
import pandas as pd
import os
import json
import io

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/"
extractions_path = "cosmos-and-extractions-jsons-for-3-papers"
annotated_files = ["data-sars-double.json", "data_modeling_covid_italy.json", "data-response-to-covid-19-was-italy-unprepared.json"]
extraction_files =["extractions_sarsdouble.json", "extractions_modeling_covid_italy--COSMOS-data.json","extractions_response-to-covid-19-was-italy-unprepared--COSMOS-data.json"]
paper_names = ["sarsdouble.pdf", "modeling_covid_italy.pdf", "response-to-covid-19-was-italy-unprepared.pdf"]
ann_extr_file_pairs = {}
for name,ann, extr in zip(paper_names,annotated_files, extraction_files):
  ann_extr_file_pairs[name] = [ann, extr]

In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/"

filenames = ["sarsdouble_all.csv", "modeling_covid_italy_all.csv", "response-to-covid-19-was-italy-unprepared_all.csv"]
events_filenames = {"sarsdouble_all.csv":"sarsdouble_events.csv", "modeling_covid_italy_all.csv":"modeling_covid_italy_events.csv", 
                    "response-to-covid-19-was-italy-unprepared_all.csv":"response_to_covid_19_was_italy_unprepared_events.csv"}


In [98]:
def get_distances(event_values:[], df:pd.DataFrame,  ctx:str, contexts_results, count):
  results = []
  event_ctx = []
  event_doc_ctx = []
  for val in event_values:
    doc_id = val[0]
    #print(val)
    sentence_num, doc_sent_linear_order, linear_order = val[6:]
    #print(doc_id, sentence_num, doc_sent_linear_order, linear_order)
    sentences = df[df['sno']<=doc_sent_linear_order][['doc_id','sentence_id','sentence_text', 'sno']].values.tolist()
    distances = {}
    for sentence in sentences:
      #print(ctx, sentence[2])
      if ctx != "NONE" and ctx in sentence[2].lower():
        #print("context "+ctx,sentence[2], doc_sent_linear_order, sentence[3], doc_sent_linear_order - sentence[3])
        distances[sentence[3]] = doc_sent_linear_order - sentence[3]
        if doc_sent_linear_order - sentence[3] == 0:
          count += 1

    sorted_distances = dict(sorted(distances.items(), key=lambda x:x[1]))
    top_3 = nsmallest(3, sorted_distances, key=sorted_distances.get)
    if len(top_3)>0:
      print(top_3[0], sorted_distances[top_3[0]], val[2])
      dist = sorted_distances[top_3[0]]
      doc = df[df['sno']==top_3[0]]
      event_ctx.append({'event_id':val[1], "ctx":ctx, "dist":dist})
      event_doc_ctx.append({'event_id':val[1], 'event':val[2], "ctx":ctx, "dist":dist,"sentence_text":doc["sentence_text"].values[0] , "tags":",".join(doc["tags"].values[0].split(" ")), 
                          "entities":",".join(doc["entities"].values[0].split(" ")), "chunks":",".join(doc["chunks"].values[0].split(" ")) })
      contexts_results[ctx] = {'doc_id':val[0], 'event_id':val[1], 'context':ctx,
      'event':val[2], 'sentence_text':val[3], 'pg_num':val[4], 'blk_id':val[5],
      "sentence_id":sentence_num, "doc_sent_linear_order":doc_sent_linear_order, "linear_order":linear_order, "distance": sorted_distances, 
      "dist": dist, "sno":top_3[0]}
      results.append({"ctx":ctx,"doc_sent_linear_order":doc_sent_linear_order, "linear_order":linear_order,"dist": dist, "sno":top_3[0]})
    else:
      top_3.append( -1)
      dist = -1    
    
  return contexts_results, count, results, event_ctx, event_doc_ctx

In [93]:
events_df

,Unnamed: 0,doc_id,annotated_page_num,para_num,event_id,event,locationContext,sentence_text,temporalContext,explanation,...,blk_id,sentence_id,doc_sentence_count,doc_sent_linear_order,tags,entities,chunks,linear_order,location_contexts,temporal_contexts
0,1916,-1949274406,2.0,3.0,E:-2045881973,"2005, the WHO issued",italy,"In,2005,,,the,WHO,issued,a,list,of,recommendat...",2005,NaN,...,4,3,13,115,"IN,CD,,,DT,WP,VBD,DT,NN,IN,NNS,-LRB-,WP,,,CD,-...","O,DATE,O,O,ORGANIZATION,O,O,O,O,O,O,O,O,DATE,O...","B-PP,B-NP,O,B-NP,I-NP,B-VP,B-NP,I-NP,B-PP,B-NP...",1917,[italy],[2005]
1,2102,-1949274406,2.0,2.0,E:-648141576,new National Preparedness Plan (PanFlu 2021-20...,italy,"More,recently,the,Ministry,of,Health,developed...","more recently,2021–2023","not a citation, but a policy called PanFlu 202...",...,4,10,13,122,"RBR,RB,DT,NNP,IN,NNP,VBD,DT,JJ,NNP,NN,NN,-LRB-...","O,DATE,O,ORGANIZATION,ORGANIZATION,ORGANIZATIO...","B-ADVP,I-ADVP,O,O,B-PP,B-NP,B-VP,B-NP,I-NP,I-N...",2103,[italy],"[more recently, 2021–2023]"
2,2156,-1949274406,3.0,3.0,E:1899714711,new plan includes an explicit monitoring funct...,italy,"Furthermore,,,the,new,plan,includes,an,explici...",2005,NaN,...,4,12,13,124,"RB,,,DT,JJ,NN,VBZ,DT,JJ,NN,NN,IN,DT,NN,IN,DT,N...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,DATE,O,O","B-ADVP,O,B-NP,I-NP,I-NP,B-VP,B-NP,I-NP,I-NP,I-...",2157,[italy],[2005]
3,1519,-1810507189,8.0,2.0,E:-765456869,those affected by chronic conditions,italy,"5,The,shift,to,digital,care,During,the,COVID-1...","covid-19 health crisis,2020",NaN,...,2,1,15,93,"CD,DT,NN,TO,JJ,NN,IN,DT,NN,NN,NN,,,IN,NNS,VBD,...","NUMBER,O,O,O,O,O,O,O,MISC,O,O,O,O,O,O,O,O,O,O,...","B-NP,B-NP,I-NP,B-PP,B-NP,I-NP,B-PP,B-NP,I-NP,I...",1520,[italy],"[covid-19 health crisis, 2020]"
4,1588,-1810507189,8.0,2.0,E:485826745,"reimbursement codes, which caused the occurren...",italy,"A,temporary,model,to,ease,organisational,aspec...","covid-19 crisis,2020",NaN,...,2,5,15,97,"DT,JJ,NN,TO,VB,JJ,NNS,IN,DT,NN,IN,NN,NNS,IN,DT...","O,O,TITLE,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,OR...","B-NP,I-NP,I-NP,B-VP,I-VP,B-NP,I-NP,B-PP,B-NP,I...",1589,[italy],"[covid-19 crisis, 2020]"
5,1618,-1810507189,8.0,3.0,E:-1225991413,"General Data Protection Regulation (GDPR),",italy,"The,COVID-19,crisis,boosted,the,digitisation,p...",covid-19 crisis,NaN,...,2,8,15,100,"DT,NN,NN,VBD,DT,NN,NN,IN,NNP,,,NNS,TO,DT,NN,IN...","O,MISC,O,O,O,O,O,O,COUNTRY,O,O,O,O,O,O,O,O,O,O...","B-NP,I-NP,I-NP,B-VP,B-NP,I-NP,I-NP,B-PP,B-NP,O...",1619,[italy],[covid-19 crisis]
6,1680,-1810507189,8.0,3.0,E:-1002533524,it led to further investments in technological...,italy,"Second,,,it,led,to,further,investments,in,tech...",covid-19 crisis,NaN,...,2,11,15,103,"RB,,,PRP,VBD,TO,JJ,NNS,IN,JJ,NN,JJ,TO,VB,DT,NN...","ORDINAL,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","B-ADVP,O,B-NP,B-VP,B-PP,B-NP,I-NP,B-PP,B-NP,I-...",1681,[italy],[covid-19 crisis]
7,1198,-1675118546,11.0,1.0,E:-1325102996,Current government instructions are based on a...,"italy,northern regions,rome","Current,government,instructions,are,based,on,a...",covid-19 outbreak,NaN,...,2,1,9,78,"JJ,NN,NNS,VBP,VBN,IN,DT,JJ,JJ,NN,NN,.","DATE,O,O,O,O,O,O,O,O,O,O,O","B-NP,I-NP,I-NP,B-VP,I-VP,B-PP,B-NP,I-NP,I-NP,I...",1199,"[italy, northern regions, rome]",[covid-19 outbreak]
8,1213,-1675118546,11.0,1.0,E:-468736132,daily rise of COVID-19 cases and related death...,italy,"At,the,time,of,writing,,,Italy,was,still,exper...",covid-19 pandemic,NaN,...,2,2,9,79,"IN,DT,NN,IN,VBG,,,NNP,VBD,RB,VBG,DT,JJ,NN,IN,N...","O,O,DATE,O,O,O,COUNTRY,O,O,O,O,SET,O,O,MISC,O,...","B-PP,B-NP,I-NP,B-PP,B-VP,O,B-NP,B-VP,I-VP,I-VP...",1214,[italy],[covid-19 pandemic]
9,1294,-1675118546,11.0,1.0,E:-1356809400,"2019, the Italian government started",italy,"In,2019,,,the,Italian,government,started,to,re...",2019,NaN,...,2,6,9,83,"IN,CD,,,DT,JJ,NN,VBD,TO,VB,DT,NN,IN,DT,NNP,,,V...","O,DATE,O,O,NATIONALITY,O,O,O,O,O,O,O,O,O,O,O,O...","B-PP,B-NP,O,B-NP,I-NP,I-NP,B-VP,I-VP,I-VP,B-NP...",1295,[italy],[2019]


In [94]:
filenames = ["sarsdouble_all.csv", "modeling_covid_italy_all.csv", "response-to-covid-19-was-italy-unprepared_all.csv"]
events_filenames = {"sarsdouble_all.csv":"sarsdouble_events.csv", "modeling_covid_italy_all.csv":"modeling_covid_italy_events.csv", 
                    "response-to-covid-19-was-italy-unprepared_all.csv":"response_to_covid_19_was_italy_unprepared_events.csv"}

loc_tf = {}
# with pd.ExcelWriter(os.path.join(path, "summarized_context_counter.xlsx")) as writer:
from heapq import nsmallest

print("\n**********************************************************************************\n")
print("The annotated extractions file name %s " %(filenames[2]))
df = pd.read_csv(os.path.join(path, filenames[2]), index_col=False)
count = 0
words = df['text'].to_list()
sentences = []
for sent in df['text'].to_list():
  sentences.extend(sent.split("\n"))


**********************************************************************************

The annotated extractions file name response-to-covid-19-was-italy-unprepared_all.csv 


In [99]:
all_results = {filename: {} for filename in filenames}

for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)

  words = df['sentence_text'].to_list()
  sentences = []
  for sent in df['text'].to_list():
    sentences.extend(sent.split("\n"))

  events_df = pd.read_csv(os.path.join(path, events_filenames[filename]), index_col=False)
  events_df.fillna("")
  events_df['locationContext'] = events_df['locationContext'].replace({"^'|'$": ""}, regex=True)
  # events_df['temporalContext'] = events_df['temporalContext'].replace({"^'|'$": ""}, regex=True)
  events_df['locationContext'] = events_df['locationContext'].str.replace('nan', "")
  events_df['locationContext'] = events_df['locationContext'].str.replace('\'','')
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('nan', "")
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('\'','')
  
  events_lctx = []
  events_doc_tctx = []
  events_doc_lctx = []
  events_tctx = []
  location_contexts = []
  temporal_contexts = []
  events_df["locationContext"] = events_df["locationContext"].apply(lambda x: str(x).replace("nan", "NONE").lower()
                                if type(x) == float else x.lower())
  events_df["temporalContext"] = events_df["temporalContext"].apply(lambda x: str(x).replace("nan", "NONE").lower()
                                if type(x) == float else x.lower())
  events_df["location_contexts"] = events_df["locationContext"].apply(lambda x: str(x).replace("nan", "NONE").lower().split(",") 
                                if type(x) == float else x.lower().split(","))
  events_df["temporal_contexts"] = events_df["temporalContext"].apply(lambda x: str(x).replace("nan", "NONE").lower().split(",") 
                                if type(x) == float else x.lower().split(","))
  for context in events_df["location_contexts"].tolist():
    if context != "NONE" :
      location_contexts.extend(context) 
  for context in events_df["temporal_contexts"].tolist():
    if context != "NONE" :
      temporal_contexts.extend(context)
  ulocation_contexts = list(set(location_contexts))
  utemporal_contexts = list(set(temporal_contexts))
  print(len(location_contexts), len(set(location_contexts)))
  print(len(temporal_contexts), len(set(temporal_contexts)))
  all_contexts = ulocation_contexts + utemporal_contexts
  lcontexts = {}
  tcontexts={}
  for ctx in ulocation_contexts:
    lcontexts[ctx] = events_df[events_df["locationContext"].str.contains(ctx)][['doc_id', 'event_id',
       'event', 'sentence_text', 'pg_num', 'blk_id', 'sentence_id', 'doc_sent_linear_order', 'linear_order']].values.tolist()
  for ctx in utemporal_contexts:
    tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',
       'event', 'sentence_text', 'pg_num', 'blk_id', 'sentence_id', 'doc_sent_linear_order', 'linear_order']].values.tolist()
  lcontexts_results = {}
  tcontexts_results = {}
  ldist_res = []
  tdist_res = []
  for ctx, values in lcontexts.items():
    lcontexts_results[ctx] = []
    #print("context ", ctx)
    contexts_results, count, results, event_ctx, event_doc_ctx = get_distances(values, df, ctx, lcontexts_results, count)
    lcontexts_results[ctx] = contexts_results
    ldist_res.extend(results)
    events_lctx.extend(event_ctx)
    events_doc_lctx.extend(event_doc_ctx)
  for ctx, values in tcontexts.items():
    tcontexts_results[ctx] = []
    contexts_results, count, results, event_ctx, event_doc_ctx = get_distances(values, df, ctx, tcontexts_results, count)
    tcontexts_results[ctx] = contexts_results
    tdist_res.extend(results)
    events_tctx.extend(event_ctx)
    events_doc_tctx.extend(event_doc_ctx)
    all_results[filename] = {"loc_results":ldist_res, "temp_results": tdist_res, "l_event_ctx":events_lctx, "t_event_ctx":events_tctx, "l_event_doc_ctx":events_doc_lctx, "t_event_doc_ctx":events_doc_tctx}


**********************************************************************************

The annotated extractions file name sarsdouble_all.csv 
295 12
853 31
53 17 set of coronavirus mediated epidemics that affected pigs
53 19 It is based on the high mutation and  recombination rate of  coronaviruses [2], and on the observation
53 22 it would cause gastroenteritis
53 24 We also explore the possible situation where  viruses A and  B would be of totally different origin, but would cause an overlapping immune response of  the host
53 3 many possibilities based on virus behavior
53 6 SARS [14], a coronavirus
53 6 virus and its variant caused a double epidemic
53 6 nature of the virus that causes SARS
22 10 the so called Susceptible-Infected-Removed (SIR) model which is simple and useful when one tries to understand the propagation of many real-life epidemics
79 41 mathematical models can have several functions
261 14 Paris VI University, Dr. W.K
261 24 Our model would then be useful to enforc

<ipython-input-99-2a73f6419a40>:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',
<ipython-input-99-2a73f6419a40>:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',
<ipython-input-99-2a73f6419a40>:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',


68 438 1d, model
68 442 many infected subjects are not than 1
68 444 lack of proper situation that leads to a peak
68 449 current statistics, which are heavily subject countermeasures, alpha = gamma = 0.057, hence R0 = 0.0329,
68 449 are heavily subject countermeasures, alpha
68 449 current statistics, which are heavily subject countermeasures, alpha = gamma = 0.057, hence R0 = 0.0329,
68 449 are heavily subject countermeasures, alpha = gamma = 0.057, hence R0
68 449 current statistics, which are heavily subject countermeasures, alpha = gamma = 0.057, hence R0
68 449 are heavily subject countermeasures, alpha = gamma = 0.057, hence R0
68 449 epidemic phenomenon based on current statistics
68 173 infected Figure 3c,d
68 205 new model that predicts the course of the epiwas extended to the entire area of Lombardy
68 220 commonly used model is the SIR
68 220 One commonly used model is the SIR after a novel strain of  coronavirus, SARS-CoV-2, was identified
68 221 More complex models can ac

<ipython-input-99-2a73f6419a40>:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',


237 18 outbreak of SARS-CoV-2 has unevenly affected Italian regions
258 0 Residenza Sanitaria Assistenziale, RSA
260 0 average of 9.1%
147 0 COVID-19 crisis has also been instrumental in Italy, as in many other European countries, in fostering the adoption of already existing digital technology by shifting to, and increasing
147 10 technical and scientific committee (hereafter CTS), comprising of high level civil servants from within the Ministry of Health, the Istituto Superiore di Sanita (National Institute of Health, ISS), regional governments and the same Civil Protection Department,
171 0 detection of the first two COVID-19 cases
173 1 excess mortality of +118%
173 7 rest of the major diagnostic groups, heavily affected by the emergency
182 0 mean rate of 13.3 emergency admissions per day
182 0 incidence rate ratio, 0.70; 95% CI, 0.63 to 0.78; p < 0.001
182 3 public health care expenditure was 74%
182 6 SSN has implemented a decentralised model since the early 1990s
182 7 Livelli 

In [89]:
events_df

,Unnamed: 0,doc_id,annotated_page_num,para_num,event_id,event,locationContext,sentence_text,temporalContext,explanation,...,blk_id,sentence_id,doc_sentence_count,doc_sent_linear_order,tags,entities,chunks,linear_order,location_contexts,temporal_contexts
0,1916,-1949274406,2.0,3.0,E:-2045881973,"2005, the WHO issued",italy,"In,2005,,,the,WHO,issued,a,list,of,recommendat...",2005,NaN,...,4,3,13,115,"IN,CD,,,DT,WP,VBD,DT,NN,IN,NNS,-LRB-,WP,,,CD,-...","O,DATE,O,O,ORGANIZATION,O,O,O,O,O,O,O,O,DATE,O...","B-PP,B-NP,O,B-NP,I-NP,B-VP,B-NP,I-NP,B-PP,B-NP...",1917,[italy],[2005]
1,2102,-1949274406,2.0,2.0,E:-648141576,new National Preparedness Plan (PanFlu 2021-20...,italy,"More,recently,the,Ministry,of,Health,developed...","more recently,2021–2023","not a citation, but a policy called PanFlu 202...",...,4,10,13,122,"RBR,RB,DT,NNP,IN,NNP,VBD,DT,JJ,NNP,NN,NN,-LRB-...","O,DATE,O,ORGANIZATION,ORGANIZATION,ORGANIZATIO...","B-ADVP,I-ADVP,O,O,B-PP,B-NP,B-VP,B-NP,I-NP,I-N...",2103,[italy],"[more recently, 2021–2023]"
2,2156,-1949274406,3.0,3.0,E:1899714711,new plan includes an explicit monitoring funct...,italy,"Furthermore,,,the,new,plan,includes,an,explici...",2005,NaN,...,4,12,13,124,"RB,,,DT,JJ,NN,VBZ,DT,JJ,NN,NN,IN,DT,NN,IN,DT,N...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,DATE,O,O","B-ADVP,O,B-NP,I-NP,I-NP,B-VP,B-NP,I-NP,I-NP,I-...",2157,[italy],[2005]
3,1519,-1810507189,8.0,2.0,E:-765456869,those affected by chronic conditions,italy,"5,The,shift,to,digital,care,During,the,COVID-1...","covid-19 health crisis,2020",NaN,...,2,1,15,93,"CD,DT,NN,TO,JJ,NN,IN,DT,NN,NN,NN,,,IN,NNS,VBD,...","NUMBER,O,O,O,O,O,O,O,MISC,O,O,O,O,O,O,O,O,O,O,...","B-NP,B-NP,I-NP,B-PP,B-NP,I-NP,B-PP,B-NP,I-NP,I...",1520,[italy],"[covid-19 health crisis, 2020]"
4,1588,-1810507189,8.0,2.0,E:485826745,"reimbursement codes, which caused the occurren...",italy,"A,temporary,model,to,ease,organisational,aspec...","covid-19 crisis,2020",NaN,...,2,5,15,97,"DT,JJ,NN,TO,VB,JJ,NNS,IN,DT,NN,IN,NN,NNS,IN,DT...","O,O,TITLE,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,OR...","B-NP,I-NP,I-NP,B-VP,I-VP,B-NP,I-NP,B-PP,B-NP,I...",1589,[italy],"[covid-19 crisis, 2020]"
5,1618,-1810507189,8.0,3.0,E:-1225991413,"General Data Protection Regulation (GDPR),",italy,"The,COVID-19,crisis,boosted,the,digitisation,p...",covid-19 crisis,NaN,...,2,8,15,100,"DT,NN,NN,VBD,DT,NN,NN,IN,NNP,,,NNS,TO,DT,NN,IN...","O,MISC,O,O,O,O,O,O,COUNTRY,O,O,O,O,O,O,O,O,O,O...","B-NP,I-NP,I-NP,B-VP,B-NP,I-NP,I-NP,B-PP,B-NP,O...",1619,[italy],[covid-19 crisis]
6,1680,-1810507189,8.0,3.0,E:-1002533524,it led to further investments in technological...,italy,"Second,,,it,led,to,further,investments,in,tech...",covid-19 crisis,NaN,...,2,11,15,103,"RB,,,PRP,VBD,TO,JJ,NNS,IN,JJ,NN,JJ,TO,VB,DT,NN...","ORDINAL,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...","B-ADVP,O,B-NP,B-VP,B-PP,B-NP,I-NP,B-PP,B-NP,I-...",1681,[italy],[covid-19 crisis]
7,1198,-1675118546,11.0,1.0,E:-1325102996,Current government instructions are based on a...,"italy,northern regions,rome","Current,government,instructions,are,based,on,a...",covid-19 outbreak,NaN,...,2,1,9,78,"JJ,NN,NNS,VBP,VBN,IN,DT,JJ,JJ,NN,NN,.","DATE,O,O,O,O,O,O,O,O,O,O,O","B-NP,I-NP,I-NP,B-VP,I-VP,B-PP,B-NP,I-NP,I-NP,I...",1199,"[italy, northern regions, rome]",[covid-19 outbreak]
8,1213,-1675118546,11.0,1.0,E:-468736132,daily rise of COVID-19 cases and related death...,italy,"At,the,time,of,writing,,,Italy,was,still,exper...",covid-19 pandemic,NaN,...,2,2,9,79,"IN,DT,NN,IN,VBG,,,NNP,VBD,RB,VBG,DT,JJ,NN,IN,N...","O,O,DATE,O,O,O,COUNTRY,O,O,O,O,SET,O,O,MISC,O,...","B-PP,B-NP,I-NP,B-PP,B-VP,O,B-NP,B-VP,I-VP,I-VP...",1214,[italy],[covid-19 pandemic]
9,1294,-1675118546,11.0,1.0,E:-1356809400,"2019, the Italian government started",italy,"In,2019,,,the,Italian,government,started,to,re...",2019,NaN,...,2,6,9,83,"IN,CD,,,DT,JJ,NN,VBD,TO,VB,DT,NN,IN,DT,NNP,,,V...","O,DATE,O,O,NATIONALITY,O,O,O,O,O,O,O,O,O,O,O,O...","B-PP,B-NP,O,B-NP,I-NP,I-NP,B-VP,I-VP,I-VP,B-NP...",1295,[italy],[2019]


In [114]:
for filename in all_results.keys():
  # with pd.ExcelWriter(os.path.join(path, filename[:-8]+"_events_context_distances.xlsx")) as writer:

  t = pd.DataFrame.from_records(all_results[filename]['t_event_doc_ctx'])
  t.drop(t.index[t['dist'] == -1], inplace = True)
  t['event'] = t['event'].astype('string')
  #print(t['event'].astype('string'))
  t.to_csv(os.path.join(path, filename[:-8]+"_events_temp_context_distances.csv"), index=False)
  gp = t.groupby('dist').count().reset_index()[['dist','event']]
  gp = gp.rename(columns={"event": "Counts"})
  gp.to_csv(os.path.join(path, filename[:-8]+"_events_temp_summary_distances.csv"), index=False)

  t = pd.DataFrame.from_records(all_results[filename]['l_event_doc_ctx'])
  t.drop(t.index[t['dist'] == -1], inplace = True)

  t.to_csv(os.path.join(path, filename[:-8]+"_events_location_context_distances.csv"), index=False)
  gp = t.groupby('dist').count().reset_index()[['dist','event']]
  gp = gp.rename(columns={"event": "Counts"})
  gp.to_csv(os.path.join(path, filename[:-8]+"_events_loc_summary_distances.csv"), index=False)

In [112]:
  t.groupby('dist').count().reset_index()

,dist,event_id,event,ctx,sentence_text,tags,entities,chunks
0,0,24,24,24,24,24,24,24
1,1,6,6,6,6,6,6,6
2,2,5,5,5,5,5,5,5
3,3,3,3,3,3,3,3,3
4,4,1,1,1,1,1,1,1
5,6,2,2,2,2,2,2,2
6,7,3,3,3,3,3,3,3
7,9,1,1,1,1,1,1,1
8,10,1,1,1,1,1,1,1
9,11,1,1,1,1,1,1,1
